In [1]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = r'C:\Users\Acer\Desktop\face mask detector'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(r'C:\Users\Acer\Desktop\face mask detector\face-mask-detection-keras-master\inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')

for layer in pre_trained_model.layers:
  layer.trainable = False
  
pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
______

In [2]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (2, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [3]:
import cv2,os

data_path=r'C:\Users\Acer\Desktop\face mask detector\face-mask-detection-keras-master\dataset\train\dataset'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary

print(label_dict)
print(categories)
print(labels)

{'with mask': 0, 'without mask': 1}
['with mask', 'without mask']
[0, 1]


In [4]:
img_size=150
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            #gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(img,(img_size,img_size))
            #resizing the gray scale into 50x50, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image


In [5]:
import numpy as np

data=np.array(data)
data=data/255
data=np.reshape(data,(data.shape[0],img_size,img_size,3 ))
target=np.array(target)

from keras.utils import np_utils

new_target=np_utils.to_categorical(target)

from sklearn.utils import shuffle
data,Label = shuffle(data,new_target, random_state=2)
train_data = [data,Label]


Using TensorFlow backend.


In [18]:
history=model.fit(train_data[0],train_data[1],epochs=10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1376 samples
Epoch 1/10
1376/1376 [==============================] - 183s 133ms/sample - loss: 0.2576 - acc: 0.9084:14 
Epoch 2/10
1376/1376 [==============================] - 115s 84ms/sample - loss: 0.0404 - acc: 0.9891
Epoch 3/10
1376/1376 [==============================] - 114s 83ms/sample - loss: 0.0672 - acc: 0.9847
Epoch 4/10
1376/1376 [==============================] - 115s 83ms/sample - loss: 0.0149 - acc: 0.9964
Epoch 5/10
1376/1376 [==============================] - 115s 83ms/sample - loss: 0.0264 - acc: 0.9927
Epoch 6/10
1376/1376 [==============================] - 114s 83ms/sample - loss: 0.0199 - acc: 0.9956
Epoch 7/10
1376/1376 [==============================] - 115s 83ms/sample - loss: 0.0209 - acc: 0.9949
Epoch 8/10
1376/1376 [==============================] - 115s 83ms/sample - loss: 0.0261 - acc: 0.9942
Epoch 9/10
1376/1376 [==============================] - 115s 83

In [8]:
checkpoint_path = r"C:\Users\Acer\Desktop\face mask detector"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [9]:
import tensorflow as tf
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)
model.save_weights(checkpoint_path.format(epoch=10))

In [7]:
import cv2,os

data_path=r'C:\Users\Acer\Desktop\ml coding workshop\Mask_Datasets\train'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary

print(label_dict)
print(categories)
print(labels)

{'Mask': 0, 'No_mask': 1}
['Mask', 'No_mask']
[0, 1]


In [8]:
img_size=150
data1=[]
target1=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            #gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(img,(img_size,img_size))
            #resizing the gray scale into 50x50, since we need a fixed common size for all the images in the dataset
            data1.append(resized)
            target1.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

Exception: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:4045: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'



In [11]:
import numpy as np
data1=np.array(data1)/255.0
data1=np.reshape(data1,(data1.shape[0],img_size,img_size,3))
target1=np.array(target1)

from keras.utils import np_utils

new_target1=np_utils.to_categorical(target1)

NameError: name 'data1' is not defined

In [10]:
import tensorflow as tf
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)

model.evaluate(data1,new_target1)


NameError: name 'data1' is not defined

In [22]:
import cv2
import numpy as np

face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#source=cv2.VideoCapture(r"C:\Users\Acer\Desktop\face mask detector\face-mask-detection-keras-master\test_video.mp4")
source=cv2.VideoCapture(0)
#source=cv2.imread(r"C:\Users\Acer\Desktop\face mask detector\face-mask-detection-keras-master\group_mask2")

source.set(1,320)
source.set(2,480)


labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [ ]:
while(True):

    ret,img=source.read()
    #gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(img,1.4,5)  

    for (x,y,w,h) in faces:
    
        face_img=img[y:y+w,x:x+w]
        a=int(y)
        b=int(x)
        a=a/2
        b=b/2
        print(a)
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()

114.0
113.5
112.5
112.0
115.5
115.0
115.0
112.0
111.0
98.0
101.0
103.0
79.0
84.5
28.5
75.5
41.0
24.0
26.0
28.0
27.5
29.0
28.0
30.5
30.0
31.5
33.0
34.5
36.0
36.0
37.0
37.5
37.5
36.0
36.0
39.0
37.5
35.0
36.5
35.0
36.0
36.0
36.0
36.0
75.0
31.5
32.0
29.5
27.0
29.5
29.0
28.5
28.5
27.0
24.5
25.0
29.5
42.5
51.5
59.5
61.0
72.0
60.0
59.0
60.0
55.5
54.5
53.0
63.0
49.0
48.0
48.0
48.0
107.0
106.0
103.5
44.0
42.0
47.0
66.0
68.0
65.0
65.0
66.0
62.0
61.0
62.0
61.5
61.5
62.0
62.5
62.0
62.5
62.0
59.5
59.5
61.5
60.5
64.5
59.0
58.0
57.5
59.0
59.5
58.5
60.0
58.5
60.5
61.5
64.5
66.0
65.5
70.5
71.5
72.0
73.0
71.5
69.5
70.5
71.0
69.5
70.0
71.0
72.5
72.0
73.0
74.0
75.5
76.0
75.0
77.5
76.0
78.5
78.5
77.0
77.0
76.5
75.5
75.0
71.0
67.0
65.0
65.0
65.5
67.0
65.5
66.0
67.0
66.5
67.0
68.5
68.5
68.0
69.5
68.0
68.5
67.5
70.5
68.0
67.5
69.0
68.0
69.5
68.0
65.5
56.5
51.0
79.0


In [21]:
while(True):

    ret,img=source.read()
    #gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(img,1.3,5) 
    #cv2.imshow("live",img)

    #for (x,y,w,h) in faces:
    
        #face_img=img[y:y+w,x:x+w]
        #a=int(y)
        #b=int(x)
        #a=a/2
        #b=b/2
        #print(a)
    resized=cv2.resize(img,(150,150))
    normalized=resized/255.0
    reshaped=np.reshape(normalized,(1,150,150,3))
    result=model.predict(reshaped)

    label=np.argmax(result,axis=1)[0]
      
    #cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
    #cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
    cv2.putText(img, labels_dict[label], (x-100, y-100),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,255),2)
        
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()

error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:4045: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [57]:

x=[10]
y=[10]
w=[10]
h=[10]
cenx=[10]
ceny=[10]
centroidx=[]
centroidy=[]
while(True):

    
    ret,img=source.read()
    faces=face_clsfr.detectMultiScale(img,1.1,4)
    a=faces.shape[0]
    array=np.array([a,faces])
    
    #print(array.shape)
    #print(a)

    #for i in range(a):
        #print(i)
        #x[i],y[i],w[i],h[i]=faces[i]
        #cenx[i]=int(x[i]/2)
        #ceny[i]=int(y[i]/2)
        
    
        
        
        
    #print(centroidx)
    
        
        
        
        
        
        

AttributeError: 'tuple' object has no attribute 'shape'

In [58]:
image = cv2.imread((r"C:\Users\Acer\Desktop\face mask detector\face-mask-detection-keras-master\group_mask2")
faces=face_clsfr.detectMultiScale(image,1.3,5)  
cv2.imshow("Results", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


SyntaxError: invalid syntax (<ipython-input-58-028172f8ea6e>, line 2)